In [1]:
# imports
import matplotlib.pyplot as plt
import matplotlib
import cv2
import os
import torch 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import random
import sys

import torchvision.transforms as transforms
import torchvision.datasets as datasets

sys.path.append('../')

from tqdm import tqdm

from resnet_dilate import *
from main import *

In [6]:
import easydict 
args = easydict.EasyDict({ "batch_size": 256, 
                          "epochs": 90, 
                          "data": 0, 
                          'arch':'resnet18',
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0,
                          'print_freq':2500,
                          'workers' : 12})

In [7]:
args.workers

12

In [3]:
device = 'cuda'
model = resnet18(pretrained=False)
model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)

In [4]:
# Data loading code
data_dir = '../ILSVRC/Data/CLS-LOC/'
traindir = os.path.join(data_dir, 'train')
valdir = os.path.join(data_dir, 'val')

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
val_dataset = datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))

In [6]:
train_sampler = None
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, shuffle=(train_sampler is None),
    num_workers=args.workers, pin_memory=True, sampler=train_sampler)

In [7]:
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=args.batch_size, shuffle=False,
    num_workers=args.workers, pin_memory=True)

In [8]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (auen): AutoEncoder(
    (conv1): Conv2d(3, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2), bias=False)
    (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (gelu): GELU()
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (upsample1): ConvTra

In [9]:
best_acc1 = 0
acc1 = 0
train_loss = []
val_acc = []
for epoch in range(args.start_epoch, args.epochs):
    start_time = time.time()
    adjust_learning_rate(optimizer, epoch, args)

    # train for one epoch
    epoch_loss = train_auen(train_loader, model, criterion, optimizer, epoch, args)

    # evaluate on validation set
    acc1 = validate_auen(val_loader, model, criterion, args)  
    
    train_loss.append(epoch_loss)
    val_acc.append(acc1)
    print('************train_loss {} val_acc {}*************'.format(epoch_loss, acc1))
    
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

#     if not args.multiprocessing_distributed or (args.multiprocessing_distributed
#             and args.rank % ngpus_per_node == 0):
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': args.arch,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    print('epoch time', time.time() - start_time)

/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][   0/5005]	Time  2.987 ( 2.987)	Data  2.407 ( 2.407)	Loss 8.2495e+00 (8.2495e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.39 (  0.39)
Epoch: [0][3000/5005]	Time  0.332 ( 0.346)	Data  0.000 ( 0.001)	Loss 4.4154e+00 (5.4483e+00)	Acc@1  13.67 (  6.82)	Acc@5  35.16 ( 18.05)
Test: [   0/1565]	Time  0.345 ( 0.345)	Loss 3.2756e+00 (3.2756e+00)	Acc@1  34.38 ( 34.38)	Acc@5  53.12 ( 53.12)
 * Acc@1 20.695 Acc@5 44.294
************train_loss None val_acc 20.694753646850586*************
epoch time 1805.364753961563
Epoch: [1][   0/5005]	Time  1.889 ( 1.889)	Data  1.770 ( 1.770)	Loss 3.7772e+00 (3.7772e+00)	Acc@1  24.22 ( 24.22)	Acc@5  47.66 ( 47.66)
Epoch: [1][3000/5005]	Time  0.357 ( 0.344)	Data  0.000 ( 0.001)	Loss 3.4864e+00 (3.6296e+00)	Acc@1  28.12 ( 26.45)	Acc@5  52.34 ( 50.28)
Test: [   0/1565]	Time  0.343 ( 0.343)	Loss 1.4319e+00 (1.4319e+00)	Acc@1  75.00 ( 75.00)	Acc@5  84.38 ( 84.38)
 * Acc@1 31.777 Acc@5 57.925
************train_loss None val_acc 31.777233123779297*************
epoch

KeyboardInterrupt: 

In [ ]:
weight = torch.tensor([[ 1.6376, -0.2284,  0.0860],
        [ 0.1055,  1.9402, -0.0046],
        [-0.0257, -0.2146,  1.5572]], device='cuda:0', requires_grad=True).detach()

In [ ]:
torch.matmul(weight,  torch.inverse(weight))

In [ ]:
weight

In [ ]:
torch.inverse(weight)

In [ ]:
torch.inverse(model.LT.detach())

In [ ]:
path = '/home/Dataset/scl/patch_images/2021.01.15/LBC521-20210112(1)/LBC521-20210112(1)_1006.png'

In [ ]:
# img = cv2.imread(path)
# img = np.resize(img, (256,256,3))
# img = torch.tensor(img)/255.
# img.shape
img = torch.randn(2,256,256,3)

In [ ]:
weight = nn.Parameter(torch.eye(3, dtype=torch.float), requires_grad=True)
weight

In [ ]:
img[0][0:5]

In [ ]:
res = torch.matmul(img, weight)
# res[0]
res.shape

In [ ]:
res.mean().backward()

In [ ]:
weight.grad